# Functions

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
import itertools
import pickle

In [2]:
#get soup of html from url
def getSoup(url):
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html")
    except:
        print("except")
        time.sleep(5)
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html")
    return soup

In [3]:
#sponsors = getSoup("https://www.aps.org/membership/units/")
#sponsors

In [4]:
def getLink(soup, regex, baseurl, type_): #get list of links
    link_list = []
    
    if type_ == 'string':
        text_link = soup.find_all('a', href = True, string = re.compile(regex, re.IGNORECASE))
        for t in text_link:
            link = t['href']
            if not re.search(r'^http', link):
                link = baseurl + link  
            link_list.append(link)
            
    if type_ == 'link': 
        for l in soup.find_all('a', attrs = {'href': re.compile(regex, re.IGNORECASE)}):
            link = l.get('href')
            if not re.search(r'^http', link):
                link = baseurl + link #add baseurl if link does not start with http
            link_list.append(link)
    
    return link_list

In [23]:
#get information in links in list by inputter links and regular expression
def LinkInfo(linkList, regex):
    infoList = []
    for i in range(0, len(linkList)):
        infoList.append(re.findall(regex, linkList[i])[0])
    return infoList

In [6]:
#get information about link by inputting a link and regular expression --> get info for link, not list of links
#def linkInfo(link, regex):
#    info = re.findall(regex, link)
#    return info

In [7]:
def yearCheck(yearList): #check number of digits in yearList, and change to only 2 digits
    for y in yearList: #this can be done smarter
        if len(y) == 4:
            yearList[yearList.index(y)] = y[-2:]
    return yearList

In [8]:
def getText(soup, regex, find_type, attribute): #attribute as string
    text_list = []
    
    if attribute == 'text':
        for t in soup.find_all(find_type, string = re.compile(regex, re.IGNORECASE)):
            text_list.append(t.text)
    
    else:
        for t in soup.find_all(find_type, attrs = {attribute: re.compile(regex, re.IGNORECASE)}):
            text_list.append(t.text)
 
    return text_list


In [9]:
def getYearLinks(url, baseurl): #get links for each year and return as dictionary
    soup = getSoup(url) 
    links = getLink(soup, r'^/meetings/baps/archives', baseurl, 'link')
    years = yearCheck(LinkInfo(links, r'\d+'))  
    
    yearLinks_dict = dict(zip(years, links))
    yearLinks_dict = {key:[val] for key, val in yearLinks_dict.items()}
    
    return yearLinks_dict

In [10]:
def removeKeys(dict_): #remove keys with no content
    for key in dict_.copy():
        if len(dict_[key]) == 0: #if no links
            dict_.pop(key)
            
    return dict_

In [199]:
def getMeetingLinks(dict_, regex1, regex2, baseurl, type1, type2): #takes an dict of links, and find the links of those pages(with regex). With 2 possibilities of regex and types
    meeting_dict = {}
    for key in dict_:
        print(key)
        linkList = []
        for i in tqdm(dict_[key]):
            soup = getSoup(i)
            links = getLink(soup, regex1, baseurl, type1)
            linkList += links

            if not links: #if link not found, use regex2
                links = getLink(soup, regex2, baseurl, type2)
                linkList += links
        
        if linkList:
            meeting_dict[key] = linkList
        
    meeting_dict = removeKeys(meeting_dict)
    
    return meeting_dict

In [12]:
def getMeetings(link, regex1, regex2, baseurl, type1, type2):#getMeetingLinks, but from link instead of dict
    soup = getSoup(link)
    links = getLink(soup, regex1, baseurl, type1)
    linkList = links

    if not links:
        links = getLink(soup, regex2, baseurl, type2)
        linkList = links
    return linkList

In [128]:
def checkLink(link, year, url):
    if link:
        soup = getSoup(link[0])
        if not soup.findAll('title'): #if no title in the soup
            link = [re.sub(r'MAR\d+', r'MAR' + year, url)] #substitute MAR part of link with MAR and the current year
    else: #if no link
        link = [re.sub(r'MAR\d+', r'MAR' + year, url)]  #substitute MAR part of link with MAR and the current year
        
    return link

In [129]:
def getBaseLink(link_dict, baseurl1, baseurl2, bulletinURL):
    Base_dict = {}
    for key in link_dict: #loop through years
        soup = getSoup(link_dict[key][0]) #get soup of link_dict of year = key
        if getText(soup, r'Bulletin', 'title', 'text'):
            link = link_dict[key]
            
        elif getLink(soup, r'abstracts', baseurl1, 'string'):
            link = getLink(soup, r'abstracts', baseurl1, 'string')
            
        elif getLink(soup, r'Bulletin Home', baseurl1, 'string'):
            link = getLink(soup, r'Bulletin Home', baseurl1, 'string')
            
        else: 
            link = getLink(soup, r'^scientific', baseurl2, 'string') 
            if link:
                if not re.findall(r'html$', link[0]):
                    link = [link[0] + "Program.html"]  #if these scientific links doesn't end at html, then add Program.html
        
        #check if link works 
        link = checkLink(link, key, bulletinURL)
        
        if link:
            Base_dict[key] = link
            
            
    Base_dict = removeKeys(Base_dict)
            
    return Base_dict     

In [55]:
def saveSoupFromList(list_, Folder, category, key):
    for i in tqdm(list_):
        soup = getSoup(i)
        #save soup
        text_file = open(Folder + "/"+ category + key + "_" + str(list_.index(i)) + ".txt", "w")
        text_file.write(str(soup)) #write string to file
        text_file.close()

In [16]:
def saveSoup(list_dict, Folder, category): #for calling saveSoupFromList. Input is dictionary
    for key in list_dict:
        print(key)
        saveSoupFromList(list_dict[key], Folder, category, key)

In [17]:
def getAllSessions(dict_, regex, baseurl, type_, Folder, category): #get all session links, and call saveSoupFromList
    for key in dict_:
        for i in tqdm(dict_[key]):
            soup = getSoup(i)
            links = getLink(soup, regex, baseurl, type_)
            saveSoupFromList(links, Folder, category, key)

In [18]:
def selectFirstLink(dict_): #input dict with list, and choose first element
    filtered_dict = {}
    for key in dict_:
        filtered_dict[key] = [dict_[key][0]]
    
    return filtered_dict

In [19]:
def removeDuplicates(dict_): #remove duplicates from list. Dictionary as input
    filtered_dict = {}
    for key in dict_:
        filtered_dict[key] = list(set(dict_[key]))
    return filtered_dict

In [157]:
def chooseLink(linkList, regex): #chooses one link based on regex
    sess = []
    for i in linkList:
        find = re.findall(regex, i) #get the session name
        sess += list(map(str.upper, find)) #convert to upper case
    session, index = np.unique(sess, return_index = True)
     
    return list(np.array(linkList)[index])

In [180]:
def getSubsessions(dict_):
    save_dict = {}
    subSessions = {}
    invitedLinks = {}
    chairLink = {}
    sessionLinks = {}
    
    for key in dict_:
        if re.findall(r'html$', dict_[key][0]): #if link ends with html
            linkList = []
            save_dict[key] = [dict_[key][0]]
            html_baseurl = re.findall(r'(.*)Program.html$', dict_[key][0])[0]
            links = getMeetings(dict_[key][0], r'sess', None, html_baseurl, 'link', None) #get link
            
            if not links: 
                soup = getSoup(dict_[key][0])
                links = getLink(soup, r'html$', html_baseurl, 'link') #save all links for 94
                subSessions[key] = links #for saving 
            
            sessions = chooseLink(links, r'Sess(\w)') 
            subSessionList = []
            for i in sessions:
                subLinks = getMeetings(i, r'\#S*', None, html_baseurl, 'link', None)
                #get links for sublinks
                if not subLinks:
                    subLinks = getMeetings(i, r'S\d+\.html$', None, html_baseurl, 'link', None)
                    if subLinks:
                        if len(re.findall(r'(\d+)\.html$', subLinks[0])[0]) == 4:
                            subSessionList += chooseLink(subLinks, r'S\w.*\d{2}\.html$')
                        else:
                            subSessionList += chooseLink(subLinks, r'/S(.*)\d{3}\.html$')
                else:
                    subSessionList += chooseLink(subLinks, r'\#S\w(.*)\.') #choose subSession links (on deepest level)
            if not sessions: #if sessions don't exist(94
                subSessions[key] = links
                
            if subSessionList:
                subSessions[key]= subSessionList
            
        else:
            invitedLinks[key] = getMeetings(dict_[key][0], r'invit', r'^Invited Speaker', invite_baseurl, 'link', 'string')
            chairLink[key] = [getMeetings(dict_[key][0], r'Chair', None, invite_baseurl, 'string', None)[0]]
            sessionLinks[key] = getMeetings(dict_[key][0], r'session', r'Session Index', invite_baseurl, 'link', 'string') #overview of all sessions
            
            if not invitedLinks[key]:
                invitedLinks[key] = [re.sub(r'\d+', key, invitedLinks['23'][0])]
        
    
    return save_dict, subSessions, invitedLinks, chairLink, sessionLinks

# Webscraping

In [24]:
url = "https://www.aps.org/meetings/baps/index.cfm" #meetings archive
baseurl = "https://www.aps.org"
invite_baseurl = "https://meetings.aps.org"
bulletin_url = "https://meetings.aps.org/Meeting/MAR23/Content/4348" #Bulletin for march meeting 2023


yearLinks = getYearLinks(url, baseurl) #link for all years
meetingsDict = getMeetingLinks(yearLinks, r'Content', None, baseurl, 'link', None) #all meeting links for each year
marchMeetings = getMeetingLinks(yearLinks, r'March Meeting \d+', r'mar\d+', baseurl, 'string', 'link')#find links for march meeting
baseLinks =  getBaseLink(marchMeetings, invite_baseurl, baseurl, bulletin_url)

23


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.79it/s]


22


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.95it/s]


21


...

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.97it/s]


94


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.95it/s]


93


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.94it/s]


In [130]:
#get sponsoring units
sponsorSoup = getSoup("https://www.aps.org/membership/units/")
sponsoringUnits = getText(sponsorSoup, 'engage', 'a', 'href')

In [92]:
#save list with sponsoring units
file = open('Files/sponsoringUnits.txt','w')
for item in sponsoringUnits:
    file.write(item+"\n")
file.close()

In [41]:
with open('Files/Dictionary/baselinks.pkl', 'wb') as fp:
    pickle.dump(baseLinks, fp)

In [181]:
#get URL for overview of sessions, subsessions, invited speakers, chair Index and sessions
saveDict, subSessions, invitedLinks, chairLinks, sessionLinks = getSubsessions(baseLinks) 

In [311]:
#save page with invited speakers and chair index
saveSoup(saveDict, 'Files/Pages', 'Pages')
saveSoup(invitedLinks, 'Files/InvitedSpeakers', 'Speakers')
saveSoup(chairLinks, 'Files/ChairIndex', 'Chair')
saveSoup(subSessions, 'Files/Sessions', 'Session')

04


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.95s/it]


03


100%|█████████████████████████████████████████████| 1/1 [00:02<00:00,  2.05s/it]


02


...

100%|█████████████████████████████████████████| 683/683 [07:47<00:00,  1.46it/s]


95


100%|█████████████████████████████████████████| 230/230 [02:38<00:00,  1.45it/s]


94


100%|█████████████████████████████████████████| 497/497 [06:00<00:00,  1.38it/s]


In [200]:
#get all links for sessions(not subsections)
sessions = getMeetingLinks(sessionLinks, r'Virtual', None, invite_baseurl, 'link', None) #all sessions in dict

23


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


22


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.87s/it]


21


...

100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.13s/it]


06


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.19s/it]


05


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.17s/it]


In [203]:
filtered_sessions = removeDuplicates(sessions) #links for all sessions(not subsections), by removing duplicates

In [207]:
subSessions_latest = getMeetingLinks(filtered_sessions, r'session', None, invite_baseurl, 'link', None) #get subsessions

23


100%|███████████████████████████████████████████| 53/53 [02:23<00:00,  2.71s/it]


22


100%|███████████████████████████████████████████| 26/26 [02:06<00:00,  4.87s/it]


21


...

100%|███████████████████████████████████████████| 22/22 [01:40<00:00,  4.57s/it]


06


100%|███████████████████████████████████████████| 25/25 [01:51<00:00,  4.47s/it]


05


100%|███████████████████████████████████████████| 23/23 [02:03<00:00,  5.35s/it]


In [229]:
#save soup for all subsessions
saveSoup(subSessions_latest, 'Files/Presenters', 'Presenter')

23


 59%|███████████████████▉              | 7399/12597 [1:31:33<9:54:12,  6.86s/it]

except


100%|███████████████████████████████████| 12597/12597 [2:46:49<00:00,  1.26it/s]


22


...

100%|███████████████████████████████████| 11379/11379 [2:09:39<00:00,  1.46it/s]


18


 42%|█████████████▌                  | 4450/10482 [14:40:46<19:53:53, 11.88s/it]

except


KeyboardInterrupt: 

In [240]:
#to get subset of dict if necessary
subset = {k:v for k,v in subSessions_latest.items() if k in 
          ['18', '17', '16', '15', '14', '13', '12', '11', '10', '09', '08', '07', '06', '05']}
saveSoup(subset, 'Files/Presenters', 'Presenter')

18


100%|███████████████████████████████████| 10482/10482 [1:58:48<00:00,  1.47it/s]


17


  5%|█▊                                   | 475/9657 [56:45<18:17:12,  7.17s/it]

except


KeyboardInterrupt: 

In [242]:
#to get subset of dict if necessary
subset_from17 = {k:v for k,v in subSessions_latest.items() if k in 
          ['17', '16', '15', '14', '13', '12', '11', '10', '09', '08', '07', '06', '05']}
saveSoup(subset_from17, 'Files/Presenters', 'Presenter')


17


 26%|█████████▎                          | 2512/9657 [30:37<18:45:16,  9.45s/it]

except


100%|█████████████████████████████████████| 9657/9657 [2:02:08<00:00,  1.32it/s]


16


 50%|██████████████████▍                  | 4672/9351 [50:10<4:46:27,  3.67s/it]

except


 75%|███████████████████████████▋         | 7011/9351 [1:55:48<38:39,  1.01it/s]

except


KeyboardInterrupt: 

In [243]:
#to get subset of dict if necessary
subset_from16 = {k:v for k,v in subSessions_latest.items() if k in 
          ['16', '15', '14', '13', '12', '11', '10', '09', '08', '07', '06', '05']}
saveSoup(subset_from16, 'Files/Presenters', 'Presenter')

16


100%|█████████████████████████████████████| 9351/9351 [1:43:27<00:00,  1.51it/s]


15


  8%|███▏                                  | 749/9076 [07:47<1:27:38,  1.58it/s]

except


100%|█████████████████████████████████████| 9076/9076 [1:56:02<00:00,  1.30it/s]


14


 61%|█████████████████████▎             | 5542/9120 [6:03:52<3:54:55,  3.94s/it]

except


KeyboardInterrupt: 

In [ ]:
#to get subset of dict if necessary
subset_from14 = {k:v for k,v in subSessions_latest.items() if k in 
          ['14', '13', '12', '11', '10', '09', '08', '07', '06', '05']}
saveSoup(subset_from14, 'Files/Presenters', 'Presenter')


#stopped at 14, 61%, 36:59 left

14


 50%|██████████████████▌                  | 4570/9120 [52:56<3:28:06,  2.74s/it]

except


100%|█████████████████████████████████████| 9120/9120 [1:55:19<00:00,  1.32it/s]


13


 79%|███████████████████████████▌       | 6733/8545 [1:09:35<1:22:17,  2.72s/it]

except


100%|█████████████████████████████████████| 8545/8545 [1:39:22<00:00,  1.43it/s]


12


 19%|██████▉                              | 1703/9054 [19:54<1:13:20,  1.67it/s]

In [247]:
#to get subset of dict if necessary
subset_from12 = {k:v for k,v in subSessions_latest.items() if k in 
          ['12', '11', '10', '09', '08', '07', '06', '05']}
saveSoup(subset_from12, 'Files/Presenters', 'Presenter')

12


100%|█████████████████████████████████████| 9054/9054 [1:34:08<00:00,  1.60it/s]


11


100%|█████████████████████████████████████| 7200/7200 [1:18:28<00:00,  1.53it/s]


10


...

100%|█████████████████████████████████████| 7058/7058 [1:42:06<00:00,  1.15it/s]


06


  1%|▏                                | 37/6957 [7:27:43<1395:36:23, 726.04s/it]

except


KeyboardInterrupt: 

In [248]:
#to get subset of dict if necessary
subset_from06 = {k:v for k,v in subSessions_latest.items() if k in 
          ['06', '05']}
saveSoup(subset_from06, 'Files/Presenters', 'Presenter')

06


100%|█████████████████████████████████████| 6957/6957 [1:18:30<00:00,  1.48it/s]


05


100%|█████████████████████████████████████| 6408/6408 [1:09:48<00:00,  1.53it/s]
